# ImageDataGeneratorを用いたデータ拡張
https://keras.io/preprocessing/image/

In [1]:
import numpy as np
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import sklearn.preprocessing as sp
import pandas as pd

Using TensorFlow backend.


## numpy形式の画像を拡張する場合

In [2]:
def onehot_to_str(label):
    """
    ワンホットベクトル形式のラベルをカタカナ文字に変換する
    """
    dic_katakana = {"a":0,"i":1,"u":2,"e":3,"o":4,"ka":5,"ki":6,"ku":7,"ke":8,"ko":9,"sa":10,"si":11,"su":12,"se":13,"so":14}
    label_int = np.argmax(label)
    for key, value in dic_katakana.items():
        if value==label_int:
            return key
    

#画像読み込み
data = np.load("../../../../DAY1/DAY1_vr6_0_0/DAY1_vr6_0_0/4_kadai/1_data/train_data.npy")  # パスは適宜変更すること
label = np.load("../../../../DAY1/DAY1_vr6_0_0/DAY1_vr6_0_0/4_kadai/1_data/train_label.npy")  # パスは適宜変更すること


In [10]:
# add_data=np.empty((0,28,28,1))
# add_label=np.empty(0)
for t in range(data.shape[0]):
    _data = data[[t]]
    _label = label[[t]]
    _label = onehot_to_str(_label)
            
    # 軸をN,H,W,Cに入れ替え
    _data = _data.transpose(0,2,3,1)

    # ImageDataGeneratorのオブジェクト生成
    datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.1,
            zoom_range=[1,1.5],
            rotation_range=10,
            horizontal_flip=False,
            vertical_flip=False,
            fill_mode="constant",cval=255)

    # 生成後枚数
    num_image = 10

    # 生成
    g = datagen.flow(_data, batch_size=1, save_to_dir="../1_data/temp7/",
                    save_format='png', save_prefix='out_%s_from_npy_'%_label)
    for i in range(num_image):
            batches = g.next()



In [11]:
import matplotlib.pyplot as plt
from matplotlib import cm
import glob

In [12]:
l=glob.glob('../1_data/temp7/*.png')
add_label=np.empty(len(l),dtype=object)
add_data=np.empty((len(l),1,28,28))

for  i in range(len(l)):
    pixel = 28
    fname = l[i][15:]
    idx = fname.split("_")[1]
    num_image = 1 # 画像枚数
    channel = 1 # グレースケール
    photo = np.empty((num_image, channel, pixel, pixel)) # 配列初期化

    img_ = Image.open(l[i])
    img_ = np.array(img_).astype(np.float32)
    photo[0, :] = img_
    add_data[i]= np.copy(photo)
    add_label[i]= idx
    # if i%1000==0:
    #     print(i)
    
add_data.shape,add_label.shape

((27231, 1, 28, 28), (27231,))

In [13]:
df_label= pd.DataFrame(add_label)
enc=sp.OneHotEncoder(categories=[["a","i","u","e","o","ka","ki","ku","ke","ko","sa","si","su","se","so"]],sparse=False)
add_label_onehot=enc.fit_transform(df_label)

In [14]:
data.shape,label.shape

((3000, 1, 28, 28), (3000, 15))

In [15]:
integ_data=np.append(data,add_data,axis=0)
integ_label=np.append(label,add_label_onehot,axis=0)
print(integ_data.shape,integ_label.shape)
np.save("../1_data/integ_data7",integ_data)
np.save("../1_data/integ_label7",integ_label)

(30231, 1, 28, 28) (30231, 15)
